In [1]:
import ccxt
import config_API_Bybit as cab
from telegram import Bot
import asyncio

In [2]:
# Inizializza il bot di Telegram
telegram_bot = Bot(token=cab.token_telegram)
TOKEN_TELEGRAM = cab.token_telegram
CHAT_ID_TELEGRAM = cab.id_chat_mio_gruppo

In [3]:
# Inizializza l'exchange Bybit
exchange = ccxt.bybit({
    'apiKey': cab.api_key,
    'secret': cab.api_secret,
    'enableRateLimit': True,
    'options': {
        'adjustForTimeDifference': True,
    },
})

# Disabilita la modalità sandbox per l'ambiente live
exchange.set_sandbox_mode(False)

# Carica i mercati
exchange.load_markets()

def get_balance(crypto_symbol, account_type):
    """
    Recupera il saldo di una specifica criptovaluta in un determinato account.

    :param crypto_symbol: Simbolo della criptovaluta (es. 'FOXY')
    :param account_type: Tipo di account (es. 'spot', 'funding', 'unified')
    :return: Dizionario con i saldi 'free', 'used' e 'total'
    """
    try:
        # Recupera il saldo per il tipo di account specificato
        balance = exchange.fetch_balance({'type': account_type, 'recvWindow': 20000}) # 10 secondi
        # Ottieni il saldo della criptovaluta specificata
        crypto_balance = balance.get(crypto_symbol, {'free': 0, 'used': 0, 'total': 0})
        return crypto_balance
    except Exception as e:
        print(f"Errore durante il recupero del saldo per {crypto_symbol} nell'account {account_type}: {e}")
        return None

# Esempio di utilizzo
crypto = 'USDT'
account_types = ['spot', 'funding', 'unified']

for account in account_types:
    balance = get_balance(crypto, account)
    if balance:
        print(f"Saldo di {crypto} nell'account {account}:")
        print(f"  Disponibile: {balance['free']}")
        print(f"  In uso: {balance['used']}")
        print(f"  Totale: {balance['total']}\n")

InvalidNonce: bybit {"retCode":10002,"retMsg":"invalid request, please check your server timestamp or recv_window param. req_timestamp[1734722778668],server_timestamp[1734722777548],recv_window[5000]","result":{},"retExtInfo":{},"time":1734722777548}

In [3]:
def invia_messaggio_telegram(token, chat_id, messaggio):
    try:
        async def _invia():
            bot = Bot(token=token)
            await bot.send_message(chat_id=chat_id, text=messaggio)
        
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(_invia())
        else:
            loop.run_until_complete(_invia())
    except Exception as e:
        print(f"Errore nell'invio del messaggio Telegram: {e}")

invia_messaggio_telegram(TOKEN_TELEGRAM, CHAT_ID_TELEGRAM, 'ciao di prova')

In [5]:
def buy_crypto(symbol, amount, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)  # Imposta a True per il testnet
    exchange.load_markets()
    order = exchange.create_market_buy_order(symbol, amount)
    return order

In [6]:
def sell_crypto(symbol, amount, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    order = exchange.create_market_sell_order(symbol, amount)
    return order

In [7]:
def set_trailing_stop(symbol, side, amount, trailing_stop, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    params = {
        'stop_loss': trailing_stop,
    }
    order = exchange.create_order(symbol, 'market', side, amount, None, params)
    return order

In [8]:
def get_average_buy_price(symbol, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    orders = exchange.fetch_closed_orders(symbol)
    total_cost = 0
    total_amount = 0
    for order in orders:
        if order['side'] == 'buy':
            total_cost += order['cost']
            total_amount += order['filled']
    if total_amount > 0:
        return total_cost / total_amount
    else:
        return 0

In [9]:
def cancel_trailing_stop_orders(symbol, api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    exchange.load_markets()
    orders = exchange.fetch_open_orders(symbol)
    for order in orders:
        if 'trailing' in order['info']:
            exchange.cancel_order(order['id'], symbol)

In [10]:
def get_available_usdt(api_key, api_secret):
    exchange = ccxt.bybit({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    exchange.set_sandbox_mode(False)
    balance = exchange.fetch_balance()
    return balance['USDT']['free']

In [ ]:
def execute_trade():
    try:
        # Esegui operazioni di trading
        order = buy_crypto('BTC/USDT', 0.01, cab.api_key, cab.api_secret)
        # Altre operazioni...
    except Exception as e:
        error_message = f"Errore durante l'esecuzione del trade: {str(e)}"
        invia_messaggio_telegram(TOKEN_TELEGRAM, CHAT_ID_TELEGRAM, error_message)

In [ ]:
# invia_messaggio_telegram(TOKEN_TELEGRAM, CHAT_ID_TELEGRAM, 'Altra prova')